# Carga Silver - Stage Controle Ativo

Este notebook realiza a carga na silver controle ativo (stg_controle_ativo) a partir dos dados da tabela bronze controle ativo (raw_controleativo).

## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_date, trim, upper, regexp_replace, 
    when, lit, current_timestamp
)
from pyspark.sql.types import DecimalType, IntegerType

## Start Spark Session

In [2]:
# Inicializa uma SparkSession com metastore persistente
builder = (
    SparkSession.builder
    .appName("Leitura e Transformação Delta Table")
    .master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.warehouse.dir", "D:/Projetos/DataLake/spark-warehouse")
    .config(
        "javax.jdo.option.ConnectionURL",
        "jdbc:derby:;databaseName=D:/Projetos/DataLake/metastore_db;create=true"
    )
    .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado com metastore persistente!\n")


✅ Spark 3.5.7 iniciado com metastore persistente!



## Variables

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"

# Define o caminho da tabela Delta Source
delta_path_controleativo = f"{base_bronze_path}/raw_controleativo"

# Define o caminho para salvar a tabela Delta transformada
delta_path_controle_ativo = f"{base_silver_path}/stg_controle_ativo"

## Read Table

In [4]:
# Lendo a tabela Delta no DataFrame do PySpark
df_bronze = spark.read.format("delta").load(delta_path_controleativo)

print(f"[SUCESSO] Leitura da tabela Delta em: {delta_path_controleativo}")
print(f"Total de registros: {df_bronze.count()}")

[SUCESSO] Leitura da tabela Delta em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_controleativo
Total de registros: 63


## Transformation

In [5]:
# Função para converter valores decimais para DECIMAL
def parse_decimal(column_name):
    return col(column_name).cast(DecimalType(11, 2))

In [ ]:
# Transformações de dados
df_silver = df_bronze.select(
    to_date(trim(col("data")), "yyyy-MM-dd HH:mm:ss").alias("dt_operacao"),
    upper(trim(col("papel"))).alias("cd_ativo"),
    upper(trim(col("op"))).alias("cd_tipo_operacao"),
    when(trim(col("qtd")).isNotNull() & (trim(col("qtd")) != ""), 
         trim(col("qtd")).cast(IntegerType()))
    .otherwise(lit(0)).alias("qt_operacao"),
    parse_decimal("preco").alias("vl_preco_ativo"),
    parse_decimal("custo_total").alias("vl_custo_total"),
    parse_decimal("total_op").alias("vl_total_operacao"),
    parse_decimal("rateio").alias("vl_rateio"),
    parse_decimal("v_liquido").alias("vl_liquido"),
    when(trim(col("estoque")).isNotNull() & (trim(col("estoque")) != ""), 
         trim(col("estoque")).cast(IntegerType()))
    .otherwise(lit(0)).alias("qt_estoque"),
    parse_decimal("pmedio").alias("vl_pmedio"),
    parse_decimal("gan_per").alias("vl_ganho_perda"),
    parse_decimal("ir_mes").alias("ir_mes"),
    trim(col("mes_ref")).alias("mes_ref"),
    parse_decimal("vendas_mes").alias("vl_vendas_mes"),
    regexp_replace(trim(col("cpf")), r"[^\d]", "").alias("cpf"),
    trim(col("cotista")).alias("cotista"),
    trim(col("arquivo_origem")).alias("arquivo_origem"),

    # Adiciona timestamp de inserção
    current_timestamp().alias("ts_insercao")
)

+-----------+--------+----------------+-----------+--------------+--------------+-----------------+---------+----------+----------+---------+--------------+------+-------+-------------+-----------+-------+--------------------+--------------------+
|dt_operacao|cd_ativo|cd_tipo_operacao|qt_operacao|vl_preco_ativo|vl_custo_total|vl_total_operacao|vl_rateio|vl_liquido|qt_estoque|vl_pmedio|vl_ganho_perda|ir_mes|mes_ref|vl_vendas_mes|        cpf|cotista|      arquivo_origem|         ts_insercao|
+-----------+--------+----------------+-----------+--------------+--------------+-----------------+---------+----------+----------+---------+--------------+------+-------+-------------+-----------+-------+--------------------+--------------------+
| 2024-10-11|  TRBL11|               C|          1|         89.95|          1.09|            89.95|     0.02|    -89.97|         7|    91.05|          NULL|  NULL|10/2024|         0.00|36725979882|TAYNARA|Controle de Ações...|2025-11-01 13:01:...|
| 2024-1

In [7]:
# Salva o DataFrame transformado em formato Delta
df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "false").save(delta_path_controle_ativo)

print(f"[SUCESSO] DataFrame transformado gravado no Delta Lake em: {delta_path_controle_ativo}")

[SUCESSO] DataFrame transformado gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_controle_ativo


## Stop Spark Session

In [8]:
# Encerra a SparkSession
spark.stop()